# Test bot

In [ ]:
from ollama_models import ollama_models
models = ollama_models()

import os
for model in models:
    dn = f'forecast_{model}'
    os.makedirs(dn, exist_ok=True)
    print("created", dn)

hard_ids = [6614, 1482, 1337, 7811, 8734, 5531, 1454, 6633, 2622, 3054]

from load_saved_questions import load_saved_questions
questions = load_saved_questions(hard_ids)

from community_forecast import community_forecast
id_to_forecast = {question.id_of_question: community_forecast(question) for question in questions}

import pandas as pd
from flatten_dict import flatten_dict
from datetime import datetime
df = pd.DataFrame([flatten_dict(q.api_json, sep='_') for q in questions])
df['id_of_post'] = [q.id_of_post for q in questions]
df['id_of_question'] = [q.id_of_question for q in questions]
df['question_options'] = df['question_options'].apply(repr)
df['today'] = datetime.now().strftime("%Y-%m-%d")
df['crowd'] = df.apply(lambda row: id_to_forecast[row.id_of_question], axis=1)
df = df[['id_of_question', 'id_of_post', 'today', 'open_time', 'scheduled_resolve_time', 'title', 'question_description',
    'question_resolution_criteria', 'question_fine_print', 'question_type',
    'question_options', 'question_group_variable', 'question_question_weight',
    'question_unit', 'question_open_upper_bound', 'question_open_lower_bound',
    'question_scaling_range_max', 'question_scaling_range_min', 'question_scaling_zero_point','crowd']]

from gather_research_and_set_prompt import gather_research_and_set_prompt
live=True
df, rag = gather_research_and_set_prompt(df, live)

from generate_forecasts_and_update_rag import generate_forecasts_and_update_rag
from tqdm import tqdm
for model in tqdm(models):
    print("Model", model)
    try:
        df = generate_forecasts_and_update_rag(df, rag, live, model)
    except:
        print("Model run failed", model)